In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

train_data = torchvision.datasets.MNIST(root='data', train=True, transform=ToTensor(), download=True)
test_data = torchvision.datasets.MNIST(root='data', train=False, transform=ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

def train_and_evaluate(model, activation_function, num_epochs=10):

    learning_rate = 0.01
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            all_preds = []
            all_labels = []
            for images, labels in test_loader:
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            acc = accuracy_score(all_labels, all_preds)
            f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
            precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
            recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

        print(f'Epoch {epoch + 1}: Accuracy={acc:.4f}, F1 Score={f1:.4f}, Precision={precision:.4f}, Recall={recall:.4f}')

class SimpleModel(nn.Module):
    def __init__(self, activation_function):
        super(SimpleModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 256)
        self.activation = activation_function
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

activation_functions = [nn.ReLU(), nn.Sigmoid(), nn.Tanh()]

for activation_function in activation_functions:
    model = SimpleModel(activation_function)
    print(f"Model with {activation_function.__class__.__name__} Activation:")
    train_and_evaluate(model, activation_function)
    print("\n")

Model with ReLU Activation:
Epoch 1: Accuracy=0.8097, F1 Score=0.7992, Precision=0.8163, Recall=0.8040
Epoch 2: Accuracy=0.8654, F1 Score=0.8627, Precision=0.8652, Recall=0.8628
Epoch 3: Accuracy=0.8839, F1 Score=0.8817, Precision=0.8832, Recall=0.8818
Epoch 4: Accuracy=0.8949, F1 Score=0.8931, Precision=0.8934, Recall=0.8932
Epoch 5: Accuracy=0.9004, F1 Score=0.8988, Precision=0.8992, Recall=0.8989
Epoch 6: Accuracy=0.9040, F1 Score=0.9024, Precision=0.9029, Recall=0.9025
Epoch 7: Accuracy=0.9066, F1 Score=0.9051, Precision=0.9057, Recall=0.9052
Epoch 8: Accuracy=0.9096, F1 Score=0.9081, Precision=0.9086, Recall=0.9083
Epoch 9: Accuracy=0.9128, F1 Score=0.9115, Precision=0.9120, Recall=0.9116
Epoch 10: Accuracy=0.9147, F1 Score=0.9134, Precision=0.9138, Recall=0.9136


Model with Sigmoid Activation:
Epoch 1: Accuracy=0.5802, F1 Score=0.5111, Precision=0.6424, Recall=0.5663
Epoch 2: Accuracy=0.6429, F1 Score=0.6004, Precision=0.7656, Recall=0.6309
Epoch 3: Accuracy=0.7417, F1 Score=0.7

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score

# Fungsi untuk melatih model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for images, labels in train_loader:
            images = images.view(-1, 28 * 28)  # Flatten the images
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# Fungsi untuk menguji model
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 28 * 28)  # Flatten the images
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')

    return accuracy, f1, precision, recall

# Menggunakan dataset MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Konfigurasi 1: Model Dangkal
class ShallowNN(nn.Module):
    def __init__(self):
        super(ShallowNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Konfigurasi 2: Model Sedang
class MediumNN(nn.Module):
    def __init__(self):
        super(MediumNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Konfigurasi 3: Model Dalam
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Konfigurasi Fungsi Rugi 1: Cross-Entropy Loss
criterion1 = nn.CrossEntropyLoss()

# Konfigurasi Fungsi Rugi 2: Mean Squared Error (MSE) Loss
criterion2 = nn.MSELoss()

# Konfigurasi Fungsi Rugi 3: Hinge Loss (untuk SVM)
criterion3 = nn.MultiMarginLoss()

# Konfigurasi Fungsi Aktivasi 1: ReLU Activation
model1 = ShallowNN()
model2 = MediumNN()
model3 = DeepNN()

# Konfigurasi Fungsi Aktivasi 2: Sigmoid Activation
model4 = ShallowNN()
model4.fc1 = nn.Sequential(model4.fc1, nn.Sigmoid())
model5 = MediumNN()
model5.fc1 = nn.Sequential(model5.fc1, nn.Sigmoid())
model6 = DeepNN()
model6.fc1 = nn.Sequential(model6.fc1, nn.Sigmoid())

# Konfigurasi Fungsi Aktivasi 3: Tanh Activation
model7 = ShallowNN()
model7.fc1 = nn.Sequential(model7.fc1, nn.Tanh())
model8 = MediumNN()
model8.fc1 = nn.Sequential(model8.fc1, nn.Tanh())
model9 = DeepNN()
model9.fc1 = nn.Sequential(model9.fc1, nn.Tanh())

# Batch size dan optimizer
batch_size = 64
learning_rate = 0.001
optimizer1 = optim.Adam(model1.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(model2.parameters(), lr=learning_rate)
optimizer3 = optim.Adam(model3.parameters(), lr=learning_rate)

# Data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#Latih dan uji model dengan fungsi aktivasi ReLu
print("Testing Model with ReLU Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model1, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

print("Testing Model with ReLU Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model2, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

print("Testing Model with ReLU Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model3, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

#Latih dan uji model dengan fungsi aktivasi Sigmoid
print("Testing Model with Sigmoid Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model4, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

print("Testing Model with Sigmoid Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model5, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

print("Testing Model with Sigmoid Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model6, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

#Latih dan uji model dengan fungsi aktivasi Tanh
print("Testing Model with Tanh Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model7, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

print("Testing Model with Tanh Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model8, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

print("Testing Model with Tanh Activation:")
accuracy1, f1_1, precision_1, recall_1 = test_model(model9, test_loader)
print(f"Akurasi: {accuracy1:.2f}%")
print(f"F1 Score: {f1_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")

Testing Model with ReLU Activation (Konfigurasi 1):


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Akurasi: 19.08%
F1 Score: 0.1220
Precision: 0.1526
Recall: 0.1929

Testing Model with ReLU Activation (Konfigurasi 2):


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Akurasi: 10.14%
F1 Score: 0.0270
Precision: 0.0361
Recall: 0.1003

Testing Model with ReLU Activation (Konfigurasi 3):


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Akurasi: 11.43%
F1 Score: 0.0579
Precision: 0.0690
Recall: 0.1189

Testing Model with Tanh Activation (Konfigurasi 2):


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Akurasi: 7.65%
F1 Score: 0.0383
Precision: 0.0719
Recall: 0.0772

Testing Model with Tanh Activation (Konfigurasi 3):
Akurasi: 8.59%
F1 Score: 0.0266
Precision: 0.0622
Recall: 0.0860



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Kesimpulannya, menunjukkan perbandingan performa model dengan aktivasi fungsi yang berbeda. Hal ini membantu untuk memahami dan membandingkan kinerja model dengan berbagai aktivasi fungsi pada situasi yang berbeda. Setiap model dievaluasi dengan berbagai metrik dan hasil evaluasi dilihat bahwa performa model (dalam hal akurasi, F1 score, precision, dan recall) dapat bervariasi tergantung pada jenis fungsi aktivasi yang digunakan yaitu ReLu, Sigmoid, dan Tanh. Masing-masing model mendapatkan nilai akurasi sebagai berikut.
* model ReLu dengan akurasi sebesar 0.9147
* model Sigmoid dengan akurasi sebesar 0.8618
* model Tanh dengan akurasi sebesar 0.9112

Dapat simpulkan bahwa dengan menggunakan model ReLu mendapatkan hasil akurasi tertinggi sebesar 0.9147 dibandingkan Sigmoid dan Tanh.